In [1]:
############################################ Import Python Libraries Required ############################################ 
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.colors

import datetime
import os 
from os import path
import seaborn as sns
from scipy import signal

import warnings
warnings.filterwarnings('ignore')

import pickle

import statistics

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
%config InlineBackend.figure_format = 'retina'



In [2]:
import requests
os.system("rm state_wise_daily.csv")
url = """https://data.covid19india.org/csv/latest/state_wise_daily.csv"""
response = requests.get(url)


with open( "state_wise_daily.csv", 'wb') as f:
    f.write(response.content)
df  = pd.read_csv("state_wise_daily.csv",delimiter=",")

listOfColumns  = ["Date","Date_YMD","Status","KA"]
df = df[listOfColumns].copy(deep=True)
df["Date"] = pd.to_datetime(df["Date"])

## Sum all the values upto Jul-1st 2020 and add that as cumuliative total to Jul-1st 2020
## The data fitted to the model starts only from Jul-1st 2020

## Sum of COnfirmed 
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Confirmed")]
ConfirmVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Confirmed")]["KA"].index
df.loc[index,"KA"] = ConfirmVal

## Sum of COnfirmed  
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Recovered")]
RecoverVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Recovered")]["KA"].index
df.loc[index,"KA"] = RecoverVal

## Sum of Deceased 
df_temp = df[(df["Date"] <= '2020-07-01') & (df["Status"]=="Deceased")]
DeceaseVal = df_temp["KA"].cumsum().iloc[-1]
index = df.loc[(df["Date"] =='2020-07-01') & (df["Status"] == "Deceased")]["KA"].index
## -2 , HARDCODED for adjustment with the data used to fir the model ( The current data we have has 253 as Total Deceased as)
df.loc[index,"KA"] = DeceaseVal - 2   


#Copy data only from Jun-01-2020
df = df[(df["Date"] >= '2020-07-01')].copy(deep=True)

##reset index of Dataframe
df.reset_index(inplace=True,drop=True)


## get the KA columns from the dataframe
totaldata_all = df.KA
totalDays  = int(len(totaldata_all)/3)

N_dataDays = totalDays

##Declare Arrays 
actualAct = np.zeros(N_dataDays) 
actualRecov = np.zeros(N_dataDays)  
actualTot = np.zeros(N_dataDays) 
actualDes = np.zeros(N_dataDays) 

## Get the first day data
MarInitTot = totaldata_all[0]
MarInitRecov = totaldata_all[1]
MarInitDes = totaldata_all[2]

##initial Setup
actualTot[0] = MarInitTot 
actualRecov[0] = MarInitRecov 
actualDes[0] = MarInitDes 
actualAct[0] = actualTot[0] - actualRecov[0] - actualDes[0]


for idx in range(1, N_dataDays):
    actualTot[idx] = actualTot[idx-1] + totaldata_all[3*(idx)]
    actualRecov[idx] = actualRecov[idx-1] + totaldata_all[3*(idx)+1]
    actualDes[idx] = actualDes[idx-1] + totaldata_all[3*(idx)+2]
    actualAct[idx] = actualTot[idx] - actualRecov[idx] - actualDes[idx]


ActualDict = {}
ActualDict["Actual"] = actualAct
ActualDict["Deceased"] = actualDes
ActualDict["Recovered"] = actualRecov
ActualDict["Cumuliative"] = actualTot

file_name = "actual_Data.pkl"

open_file = open(file_name, "wb")
pickle.dump(ActualDict, open_file)
open_file.close()